# 分散表現を用いたGroebner基底の計算

- SymPyのPolyのように，独自のメソッドを持った新しい型（？）を作りたい

## 0521課題

- 基本的に入力出力共にDRPoly型にすること
- 正規化の手順の見直し
- Sortingの使用回数を極力減らす

**分散表現を活かした実装を！！！**

## 0601課題

- insert, copyは重いので，極力使いたくない
- add, subsの実装の改善
  - gを固定し，fをひとつずつ確認：O(mn)
  - f, gを先頭から比較し（orderの高い方を動かす），項順序の低いものとは絶対にマッチしないという性質を使って，O(m+n)で済ませる．
- reductionの改善
  - fの内容が変わると，Gの全てで再度チェックが必要
  - reductionの計算がBuchbergerAlgorithmの大部分を占めているため，丁寧に実装
  - add, subsを直接実装し，無駄を減らす．
  - fのht_gの倍単項式となる単項式t以上の項はreductionで動かないため，計算に含めない
  - 項順序がt以下の単項式たちは，add・subs参考に

## ボツの関数

In [1]:
def __DRPoly_addOrSub(f, g, key, order):
    """DRPoly型の足し算と引き算を行う関数の本体"""
    import copy
    result = []
    g_copy = copy.copy(g)
    for monom_f in f:
        j = -1
        for monom_g in g_copy:
            j += 1
            if monom_f[0] == monom_g[0]:
                if key == 'add':
                    result.append((monom_f[0], monom_f[1] + monom_g[1]))
                elif key == 'sub':
                    result.append((monom_f[0], monom_f[1] - monom_g[1]))
                g_copy.remove(monom_g)
                break
            #elif key == 'sub':
              #  g_copy[j] = ((monom_g[0], -monom_g[1]))
        else:
            result.append(monom_f)
    if key == 'sub':
        g_copy2 = copy.copy(g_copy)
        i = -1
        for m in g_copy:
            i += 1
            g_copy2[i] = (m[0], -m[1])
        g_copy = g_copy2
    result = DRPoly_zeroRemove(result + g_copy)
    return DRPoly_sorting(result, order)

In [2]:
def __DRPoly_monomMul(m1, m2):
    """DRPoly型の単項式同士の掛け算"""
    m1, m2 = m1[0], m2[0]
    m1_exp, m1_coeff = m1
    m2_exp, m2_coeff = m2
    exp_result = []
    for exp1, exp2 in zip(m1_exp, m2_exp):
        exp_result.append(exp1 + exp2)
    return [(tuple(exp_result), m1_coeff*m2_coeff)]

In [3]:
def __DRPoly_mul(f, g, order):
    """DRPoly型の掛け算(多項式＊多項式)"""
    import copy
    if type(f) == DRPoly:
        f = f.List()
    if type(g) == DRPoly:
        g = g.List()
        
    if len(f) == 1 and len(g) == 1:
        return DRPoly_monomMul(f, g)
    mul_result = []
    for monom_f in f:
        for monom_g in g:
            exp = DRPoly_monomMul([monom_f], [monom_g])[0]
            mul_result.append(exp)
    i = -1
    for m1 in mul_result:
        i += 1
        result_copy = copy.copy(mul_result)
        result_copy.remove(m1)
        for m2 in result_copy:
            if m1[0] == m2[0]:
                mul_result[i] = (m1[0], m1[1]+m2[1])
                m1 = mul_result[i]
                mul_result.remove(m2)
    mul_result = DRPoly_zeroRemove(mul_result)
    return DRPoly_sorting(mul_result, order)

In [4]:
#############################################
def __DRPoly_addOrSub(f, g, key, order):
    """DRPoly型の足し算と引き算を行う関数の本体"""
    f_original, g_original = f, g
    f_copy, g_copy = f.copy(), g.copy()
    for monom_g in g_copy:
        f_index = -1
        for monom_f in f_copy:
            f_index += 1
            if monom_f[0] == monom_g[0]:
                if key == 'sub':
                    f_copy[f_index] = ((monom_f[0]), monom_f[1] - monom_g[1])
                elif key == 'add':
                    f_copy[f_index] = ((monom_f[0]), monom_f[1] + monom_g[1])
                break
            elif DRPoly_termOrderCompare([monom_f], [monom_g], order) == [monom_g]:
                if key == 'sub':
                    f_copy.insert(f_index, ((monom_g[0]), -monom_g[1]))
                elif key == 'add':
                    f_copy.insert(f_index, monom_g)
                break
        else:
            if key == 'sub':
                f_copy.append(((monom_g[0]), -monom_g[1]))
            elif key == 'add':
                f_copy.append(monom_g)
    result = f_copy
    f, g = f_original, g_original
    return DRPoly_zeroRemove(result)
###############################################

In [5]:
def __DRPoly_normalization(f, G):
    """DRPoly型の多項式fのGでの正規形を出力"""
    break_flag = False
    all_check = True
    f_list = f.List()
    zeroTuple = tuple([0 for _ in range(len(f_list[0][0]))])
    while True:
        for g in G:
            ht_g = g.ht()
            # while文でまわす
            # itertools
            for monom_f in f.List():
                monom_f = DRPoly([monom_f], order)
                if DRPoly_monomDivisible(monom_f, ht_g):
                    hm_g = g.hm()
                    h = f - DRPoly_monomQuo(monom_f, hm_g) * g
                    f = h
                    if f.is_zero():
                        return f
                    break_flag = True
                    all_check = False
                    break
                elif DRPoly_termOrderCompare(monom_f, ht_g, order) == ht_g:
                    break_flag = True
                    all_check = False
                    break
            if break_flag:
                break_flag = False
                break
        if all_check:
            return f
        all_check = True

（↑非表示中）

# DRPoly型の各種演算の定義

## 準備の関数

In [6]:
from sympy import *
from sympy.abc import x, y, z

In [7]:
"""注意：基本的に入力出力共にDRPoly型にすること"""

'注意：基本的に入力出力共にDRPoly型にすること'

In [8]:
def Poly2DRPoly(f, order):
    """SymPyのPoly型を分散表現の形（List）に変換"""
    f_gens = f.gens
    f = f.as_expr()
    # 指定の項順序での順番に単項式を並び替える．
    f_ordered_monos = f.as_ordered_terms(order=order)
    DRPoly = []
    for m in f_ordered_monos:
        m_tuple = ()
        for v in f_gens:
            m_tuple += (degree(m, v), )
        m_tuple = (m_tuple, ) + tuple(Poly(m, f_gens).coeffs())
        DRPoly.append(m_tuple)
    return DRPoly

In [9]:
def displayDRPoly(f, gens):
    import IPython
    displayString = ''
    if len(f) == 1:
        exp, coeff = f[0]
        if exp == tuple([0 for _ in range(len(exp))]):
            return str(coeff)
        if coeff != 1 and coeff != -1:
            displayString += str(coeff) + '*'
        elif coeff == -1:
            displayString += '-'
        for v, e in zip(gens, exp):
            if e != 0:
                if e != 1:
                    displayString += str(v) + '**' + str(e) + '*'
                else:
                    displayString += str(v) + '*'
        if displayString[-1] == '*':
            return displayString[:-1]
        else: return displayString
        
    _f = f
    last_gen = gens[-1]
    if type(_f) == DRPoly:
        _f = f.List()
    expList, coeffList, termList = [], [], []
    
    for m in _f:
        exp, coeff = m
        expList.append(exp)
        coeffList.append(coeff)
    if len(_f) == 1 and all(list(expList[0])) == 0:
        return str(coeffList[0])
        
    polynomialString = ''
    for coeff, termExp in zip(coeffList, expList):
        if coeff == 1:
            polynomialString += ' + '
            if termExp == tuple([0 for _ in termExp]):
                polynomialString += '1'
        elif coeff == -1:
            polynomialString += ' - '
            if termExp == tuple([0 for _ in termExp]):
                polynomialString += '1'
        elif coeff < 0:
            polynomialString += ' - ' + str(-coeff) + '*'
        else:
            polynomialString += ' + ' + str(coeff) + '*'
        for gen, exp in zip(gens, termExp):
            if exp == 0:
                continue
            elif exp == 1:
                polynomialString += gen
            else:
                polynomialString += gen + '**' + str(exp)
            if gen != last_gen:
                polynomialString += '*'
        if polynomialString[-1] == '*':
            polynomialString = polynomialString[:-1]
    
    if polynomialString[1] == '+':
        polynomialString = polynomialString[3:]
    if polynomialString[1] == '-':
        polynomialString = '-' + polynomialString[3:]

    return polynomialString

In [10]:
def DRPoly_makeOrderMatrix(len_v, order):
    """各項順序のMatrix orderを定義"""
    if order == 'lex':
        M = eye(len_v)
    if order == 'grlex':
        M = eye(len_v).row_insert(0, Matrix([[1 for _ in range(len_v)]]))
    if order == 'grevlex':
        M = []
        for i in range(len_v):
            M.append([0 for _ in range(len_v)])
            M[i][len_v-i-1] = -1
        M = Matrix(M).row_insert(0, Matrix([[1 for _ in range(len_v)]]))
    return M

In [11]:
def DRPoly_sorting(DRPoly_f, order):
    """分散表現（List）を項順序の高い順に並び替え"""
    # Listへ変換
    if type(DRPoly_f) == DRPoly:
        DRPoly_f = DRPoly_f.List()
        
    len_v = len(DRPoly_f[0][0])
    M = DRPoly_makeOrderMatrix(len_v, order)
    
    expMatrix, coeffsList = [], []
    for vec in DRPoly_f:
        exp, coeff = vec
        expMatrix.append(list(exp))
        coeffsList.append(coeff)
    expMatrix = Matrix(expMatrix)
    
    flag = True
    while flag:
        flag = False
        for i in range(expMatrix.shape[0] -1):
            v = expMatrix.row(i) - expMatrix.row(i+1)
            v = M * v.transpose()
            for j in range(len_v):
                if v[j] < 0:
                    expMatrix.row_swap(i, i+1)
                    coeffsList[i], coeffsList[i+1] = coeffsList[i+1], coeffsList[i]
                    flag = True
                    break
                elif v[j] > 0:
                    break
                    
    expList = expMatrix.tolist()
    result = []
    for exp, coeff in zip(expList, coeffsList):
        result.append((tuple(exp), coeff))
    return result

In [12]:
def DRPolyList_sorting(G, order):
    """要素がDRPoly型のListを，HTの大きい順に並べる"""
    ht_g_list = [g.ht() for g in G]
    length = len(ht_g_list)
    for i in range(length):
        for j in range(length-1, i, -1):
            if DRPoly_termOrderCompare(ht_g_list[j], ht_g_list[j-1], order) == ht_g_list[j].List():
                ht_g_list[j], ht_g_list[j-1] = ht_g_list[j-1], ht_g_list[j]
                G[j], G[j-1] = G[j-1], G[j]

In [13]:
def DRPoly_termOrderCompare(t1, t2, order):
    """単項式2つを指定の項順序で大きい方を返す"""
    if type(t1) == DRPoly:
        _t1 = t1.List()
    else:
        _t1 = t1
    if type(t2) == DRPoly:
        _t2 = t2.List()
    else:
        _t2 = t2
    len_v = len(_t1[0][0])
    M = DRPoly_makeOrderMatrix(len_v, order)
    t1Vector = Matrix(list(_t1[0][0]))
    t2Vector = Matrix(list(_t2[0][0]))
    Vector = t1Vector - t2Vector
    compareVector = M * Vector
    for u in compareVector:
        if u > 0:
            return t1
        elif u < 0:
            return t2
        elif u == 0:
            continue
    return t1

In [14]:
#############################################
def __DRPoly_addOrSub(f, g, key, order):
    """DRPoly型の足し算と引き算を行う関数の本体"""
    f_original, g_original = f, g
    f_copy, g_copy = f.copy(), g.copy()
    for monom_g in g_copy:
        f_index = -1
        for monom_f in f_copy:
            f_index += 1
            if monom_f[0] == monom_g[0]:
                if key == 'sub':
                    f_copy[f_index] = ((monom_f[0]), monom_f[1] - monom_g[1])
                elif key == 'add':
                    f_copy[f_index] = ((monom_f[0]), monom_f[1] + monom_g[1])
                break
            elif DRPoly_termOrderCompare([monom_f], [monom_g], order) == [monom_g]:
                if key == 'sub':
                    f_copy.insert(f_index, ((monom_g[0]), -monom_g[1]))
                elif key == 'add':
                    f_copy.insert(f_index, monom_g)
                break
        else:
            if key == 'sub':
                f_copy.append(((monom_g[0]), -monom_g[1]))
            elif key == 'add':
                f_copy.append(monom_g)
    result = f_copy
    f, g = f_original, g_original
    return DRPoly_zeroRemove(result)
###############################################

In [15]:
################################################
def DRPoly_add(f, g, order):
    """DRPoly型の足し算"""
    f_plus_g = []
    len_f, len_g = len(f), len(g)
    f_index, g_index = 0, 0
    s_f_finished, is_g_finished = False, False
    
    while True:
        _f, _g = f[f_index], g[g_index]
        if _f[0] == _g[0]:
            f_plus_g.append((_f[0], _f[1]+_g[1]))
            f_index += 1
            g_index += 1
        else:
            if DRPoly_termOrderCompare([_f], [_g], order=order) == [_f]:
                f_plus_g.append(_f)
                f_index += 1
            else:
                f_plus_g.append(_g)
                g_index += 1
        if f_index == len_f or g_index == len_g:
            if f_index == len_f:
                is_f_finished = True
            if g_index == len_g:
                is_g_finished = True
            if is_f_finished and is_g_finished:
                #return f_plus_g
                break
            elif is_f_finished:
                f_plus_g += g[g_index:]
                #return f_plus_g
                break
            else:
                f_plus_g += f[f_index:]
                #return f_plus_g
                break
    return DRPoly_zeroRemove(f_plus_g)
################################################

In [16]:
################################################
def DRPoly_sub(f, g, order):
    """DRPoly型の引き算"""    
    f_plus_g = []
    len_f, len_g = len(f), len(g)
    f_index, g_index = 0, 0
    is_f_finished, is_g_finished = False, False
    
    while True:
        _f, _g = f[f_index], g[g_index]
        if _f[0] == _g[0]:
            f_plus_g.append((_f[0], _f[1]-_g[1]))
            f_index += 1
            g_index += 1
        else:
            if DRPoly_termOrderCompare([_f], [_g], order=order) == [_f]:
                f_plus_g.append(_f)
                f_index += 1
            else:
                f_plus_g.append((_g[0], -_g[1]))
                g_index += 1
        if f_index == len_f or g_index == len_g:
            if f_index == len_f:
                is_f_finished = True
            if g_index == len_g:
                is_g_finished = True
            if is_f_finished and is_g_finished:
                #return f_plus_g
                break
            elif is_f_finished:
                rem_g = g[g_index:]
                f_plus_g += [(exp, -coeff) for (exp, coeff) in rem_g]
                #return f_plus_g
                break
            else:
                f_plus_g += f[f_index:]
                #return f_plus_g
                break
    return DRPoly_zeroRemove(f_plus_g)
################################################

In [17]:
def DRPoly_mul(f, g, order):
    """DRPolyの掛け算（単項式＊多項式）"""
    """[((2, 3), 2)] * [((2, 4), -1), ((1, 0), 3)]"""
    if len(f) == 1:
        monom, poly = f.copy(), g.copy()
    else:
        monom, poly = g.copy(), f.copy()
    t2_exp, t2_coeff = list(monom[0][0]), monom[0][1]
    poly_index = -1
    for m in poly:
        poly_index += 1
        t1_exp = list(m[0])
        poly[poly_index] = (tuple([t1+t2 for t1, t2 in zip(t1_exp, t2_exp)]), t2_coeff*m[1])
    return poly

In [18]:
def DRPoly_monomQuo(f, g, order):
    """DRPoly型の単項式同士の割り算の商（割り切れないときはValueError）"""
    Quo = []
    import copy
    _f, _g = copy.copy(f), copy.copy(g)
    if type(_f) == DRPoly:
        _f = _f.List()
    if type(_g) == DRPoly:
        _g = _g.List()
    for i in range(len(_f[0][0])):
        Quo.append(_f[0][0][i] - _g[0][0][i])
    coeff = Rational(_f[0][1], _g[0][1])
    result = [(tuple(Quo), coeff)]
    return DRPoly(DRPoly_zeroRemove(result), order)
    #return DRPoly(result, order)

In [19]:
def DRPoly_monomDivisible(f, g):
    """DRPoly型の単項式同士が割り切るかどうかを調べる"""
    _f, _g = f, g
    if type(_f) == DRPoly:
        _f = _f.List()
    if type(_g) == DRPoly:
        _g = _g.List()
    f_list = list(_f[0][0])
    g_list = list(_g[0][0])
    subsList = [f_list[i] - g_list[i] for i in range(len(f_list))]
    positiveCheck = [i >= 0 for i in subsList]
    return all(positiveCheck) == True

In [20]:
def DRPoly_zeroRemove(f):
    """DRPoly型の計算を通して，[((...), 0)]が発生した場合，取り除く"""
    if type(f) == DRPoly:
        f = f.List()
    len_v = len(f[0][0])
    i = len(f)
    for monom_f in reversed(f):
        i -= 1
        if monom_f[1] == 0:
            del f[i]
    if f == []:
        return [(tuple([0 for _ in range(len_v)]), 0)]
    return f

In [21]:
def DRPoly_monomLCM(m1, m2):
    """DRPoly型の単項式のLCMを計算する"""
    order = m1.order
    if type(m1) == DRPoly:
        _m1 = m1.List()
    if type(m2) == DRPoly:
        _m2 = m2.List()
    
    t1, t2 = _m1[0][0], _m2[0][0]
    coeff_LCM = lcm(_m1[0][1], _m2[0][1])
    result = []
    for t1_exp, t2_exp in zip(t1, t2):
        result.append(max(t1_exp, t2_exp))
    return DRPoly([(tuple(result), coeff_LCM)], order)

In [22]:
def DRPoly_totalDegree(f):
    """DRPoly型のtotal degree"""
    _f = f
    if type(f) == DRPoly:
        _f = f.List()
    Max = 0
    for exp, coeff in f:
        termTotalDegree = sum(exp)
        if Max < termTotalDegree:
            Max = termTotalDegree
    return Max

In [23]:
def firstSugar(f):
    if type(f) == Poly:
        return total_degree(f)
    elif type(f) == list:
        return DRPoly_totalDegree(f)
    elif type(f) == DRPoly:
        return f.totalDegree()

In [24]:
def __DRPoly_normalForm(f, g, test_print=False):
    order = f.order
    #g_list = g.List().copy()
    g_list = g.List()
    ht_g = g.ht().List()
    hm_g = g.hm().List()
    
    return_flag = False
    flag = True
    while flag:
        flag = False
        f_list = f.List().copy()
        for monom_f in f_list:
            if DRPoly_monomDivisible([monom_f], ht_g):
                return_flag = True
                if test_print:
                    print('f : ', f)
                    print('monom_f : ', [monom_f])
                    print('hm_g : ', hm_g)
                    print('g : ', g)
                    print('monomQuo : ', DRPoly_monomQuo([monom_f], hm_g, order))
                h = f - DRPoly_monomQuo([monom_f], hm_g, order) * g
                if test_print:
                    print('result : ', h)
                    print('---------------------------')
                f = h
                if f.is_zero():
                    return return_flag, f
                flag = True
                break
            elif DRPoly_termOrderCompare(ht_g, [monom_f], order) == ht_g:
                break
    return return_flag, f

In [25]:
################################################################
def DRPoly_normalForm(f, g, test_print=False):
    order = f.order
    len_v = f.len_gens
    g_list = g.List()
    ht_g = g.ht().List()
    hm_g = g.hm().List()
    hm_g_exp, hm_g_coeff = hm_g[0][0], hm_g[0][1]
    #hm_g_exp = [hm_g_exp]
    is_divisible = False
    result_f = []
    rem_f = f.List()
    return_flag = False
    
    while True:
        rem_f_index = -1
        # ht_gの倍多項式となるfの単項式を探索（なければreturn）
        for monom_f in rem_f:
            rem_f_index += 1
            t_exp, t_coeff = monom_f
            #print(t_exp, hm_g_exp)
            compareMatrix = [exp_i - hm_g_exp_i for exp_i, hm_g_exp_i in zip(t_exp, hm_g_exp)]
            #print(compareMatrix)
            is_divisible_list = []
            for i in compareMatrix:
                if i < 0:
                    is_divisible_list.append(False)
                else:
                    is_divisible_list.append(True)
            if sum(is_divisible_list) == len(is_divisible_list):
                quo_coeff = Rational(t_coeff, hm_g_coeff)
                break
            #if DRPoly_termOrderCompare(ht_g, [monom_f], order) == ht_g:
              #  return return_flag, DRPoly(result_f + rem_f, order)
        # for文が全て回れば
        else:
            return return_flag, DRPoly(result_f + rem_f, order)
        
        return_flag = True
        
        #result_fとrem_fの更新
        if rem_f_index != 0:
            result_f += rem_f[:rem_f_index]
            rem_f = rem_f[rem_f_index:]
        #else:
          #  rem_f = f
            
        quo_exp = tuple([t_i - hm_g_i for t_i, hm_g_i in zip(t_exp, hm_g_exp)])
        quo_part = [(quo_exp, quo_coeff)]
        
        # reductionの計算
        #rem_f = (DRPoly(rem_f, order) - DRPoly(quo_part, order)*DRPoly(g, order)).List()
        
        #mul_result = g * quo_part
        mul_result = []
        for monom_g in g_list:
            monom_g_exp, monom_g_coeff = monom_g
            #print(mul_result)
            mul_result.append((tuple([t1+t2 for t1, t2 in zip(quo_exp, monom_g_exp)]), quo_coeff*monom_g_coeff))
        
        ## zero remove
        mul_result_index = 0
        len_mul_result = len(mul_result)
        while mul_result_index < len_mul_result:
            elem_exp, elem_coeff = mul_result[mul_result_index]
            if elem_coeff == 0:
                mul_result_index -= 1
                len_mul_result -= 1
                del mul_result[mul_result_index]
            mul_result_index += 1
        
        # rem_f  <- rem_f - mul_result
        #rem_f => h1, mul_result => h2
        h1_subs_h2 = []
        len_h1, len_h2 = len(rem_f), len(mul_result)
        h1_index, h2_index = 0, 0
        is_h1_finished, is_h2_finished = False, False

        while True:
            h1, h2 = rem_f[h1_index], mul_result[h2_index]
            if h1[0] == h2[0]:
                h1_subs_h2.append((h1[0], h1[1]-h2[1]))
                h1_index += 1
                h2_index += 1
            else:
                if DRPoly_termOrderCompare([h1], [h2], order=order) == [h1]:
                    h1_subs_h2.append(h1)
                    h1_index += 1
                else:
                    h1_subs_h2.append((h2[0], -h2[1]))
                    h2_index += 1
            if h1_index == len_h1 or h2_index == len_h2:
                if h1_index == len_h1:
                    is_h1_finished = True
                if h2_index == len_h2:
                    is_h2_finished = True
                if is_h1_finished and is_h2_finished:
                    break
                elif is_h1_finished:
                    rem_h2 = mul_result[h2_index:]
                    h1_subs_h2 += [(exp, -coeff) for (exp, coeff) in rem_h2]
                    break
                else:
                    h1_subs_h2 += rem_f[h1_index:]
                    break
                    
        ## zero remove
        subs_result_index = 0
        len_subs_result = len(h1_subs_h2)
        while subs_result_index < len_subs_result:
            #print(h1_subs_h2)
            elem_exp, elem_coeff = h1_subs_h2[subs_result_index]
            if elem_coeff == 0:
                del h1_subs_h2[subs_result_index]
                subs_result_index -= 1
                len_subs_result -= 1
            subs_result_index += 1
            
        rem_f = h1_subs_h2
        
        if rem_f == [] or DRPoly(rem_f, order).is_zero():
            if result_f == []:
                return True, DRPoly([(tuple([0 for i in range(len_v)]), 0)], order)
            else:
                return True, DRPoly(result_f, order)
        
        
    """
    
    return_flag = False
    flag = True
    while flag:
        flag = False
        f_list = f.List().copy()
        for monom_f in f_list:
            if DRPoly_monomDivisible([monom_f], ht_g):
                return_flag = True
                h = f - DRPoly_monomQuo([monom_f], hm_g, order) * g
                f = h
                if f.is_zero():
                    return return_flag, f
                flag = True
                break
            elif DRPoly_termOrderCompare(ht_g, [monom_f], order) == ht_g:
                break
    return return_flag, f"""
################################################################

## DRPoly本体

In [26]:
class MyError(Exception):
    pass

In [27]:
class DRPoly:
    def __init__(self, f, order='lex', sugar=None):
        if type(f) == Poly:
            self.DRPoly_f = Poly2DRPoly(f.as_poly(), order)
            self.gens = tuple([str(gen) for gen in f.gens])
            self.len_gens = len(f.gens)
        elif type(f) == list:
            self.DRPoly_f = f
            self.len_gens = len(f[0][0])
            if self.len_gens <= 5:
                gens_list = ['x', 'y', 'z', 'u', 'w']
                self.gens = tuple(gens_list[:self.len_gens])
            else:
                self.gens = tuple(['x' + str(i) for i in range(self.len_gens)])
        elif type(f) == DRPoly:
            self.DRPoly_f = f.List()
            self.len_gens = len(f.List()[0][0])
        self.order = order
        if sugar == None:
            self.sugar = firstSugar(f)
        else:
            self.sugar = sugar
    def __repr__(self):
        return 'DRPoly(' + displayDRPoly(self.DRPoly_f, self.gens) + ')'
    def __eq__(self, other):
        if isinstance(other, DRPoly):
            _other = other.List()
        elif type(other) == list:
            _other = other
        else:
            raise ValueError(('self is ' + str(type(self)), 'other is ' + str(type(other)) + '.'))
        return self.DRPoly_f == _other
    def __add__(self, other):
        return DRPoly(DRPoly_add(self.DRPoly_f, other.List(), self.order),
                      self.order, sugar=max(self.sugar, other.sugar))
    def __sub__(self, other):
        return DRPoly(DRPoly_sub(self.DRPoly_f, other.List(), self.order),
                      self.order, sugar=max(self.sugar, other.sugar))
    def __mul__(self, other):
        ##### Buchbergerでは基本的に(単項式＊多項式)なので，sugarの受け渡しもそれに基づいた．
        if len(self.DRPoly_f) == 1:
            monom, poly = self.DRPoly_f, other.List()
            monom_sugar, poly_sugar = DRPoly_totalDegree(monom), other.sugar
        elif len(other.List()) == 1:
            monom, poly = other.List(), self.DRPoly_f
            monom_sugar = DRPoly_totalDegree(monom), self.sugar
        else:
            monom_sugar, poly_sugar = 0, 0
        return DRPoly(DRPoly_mul(self.DRPoly_f, other.List(), self.order),
                      self.order, sugar=monom_sugar+poly_sugar)
    def toPositive(self):
        f_copy = self.DRPoly_f.copy()
        if self.DRPoly_f[0][1] < 0:
            result = []
            for exp, coeff in f_copy:
                result.append((exp, -coeff))
            return DRPoly(result, self.order)
        else: return DRPoly(f_copy, self.order)
    def monomQuo(self, other):
        return DRPoly(DRPoly_monomQuo(self.DRPoly_f, other.List()), self.order)
    def monomLCM(self, other):
        return DRPoly(DRPoly_monomLCM(self.DRPoly_f, other.List()), self.order)
    def totalDegree(self):
        return DRPoly_totalDegree(self.DRPoly_f)
    def List(self):
        return self.DRPoly_f
    def hm(self):
        #return [self.DRPoly_f[0]]
        return DRPoly([self.DRPoly_f[0]])
    def ht(self):
        return DRPoly([(self.DRPoly_f[0][0], 1)])
    def hc(self):
        return self.DRPoly_f[0][1]
    def is_zero(self):
        zeroTuple = [(tuple([0 for i in range(self.len_gens)]), 0)]
        return zeroTuple == self.DRPoly_f
    def normal_form(self, other):
        """fのgについてのnormal formを計算する．（入出力ともにDRPoly）"""
        return DRPoly_normalForm(self, other)

In [28]:
if __name__ == '__main__':
    f = Poly(-4*x**2*y**3 + 3*x*y**3 - 2*x**2*y**2)
    #g = Poly(-3*x**4*y - 9*x**2*y**3 + 5*x*y**2 - 1)
    g = f + Poly(x - y**2 - 5)
    t = Poly(-4*x**2*y**3)
    display(f)
    display(g)
    order = 'lex'
    F = DRPoly(f, order=order)
    F_list = F.List()
    G = DRPoly(g, order=order)
    G_list = G.List()
    T = DRPoly(t, order=order)
    T_list = T.List()
    #_1, _2, _3, _4 = F + G, F - G, F * G, DRPoly_monomQuo(F, G)
    print(t - f)
    print(T - F)

Poly(-4*x**2*y**3 - 2*x**2*y**2 + 3*x*y**3, x, y, domain='ZZ')

Poly(-4*x**2*y**3 - 2*x**2*y**2 + 3*x*y**3 + x - y**2 - 5, x, y, domain='ZZ')

Poly(2*x**2*y**2 - 3*x*y**3, x, y, domain='ZZ')
DRPoly(2*x**2*y**2 - 3*x*y**3)


In [29]:
if __name__ == '__main__':
    """DRPoly(x*y*z*u), DRPoly(1)"""
    f1 = DRPoly([((1, 1, 1, 1), 1)])
    f2 = DRPoly([((0, 0, 0, 0), 1)])
    DRPoly_termOrderCompare(f1, f2, 'grlex')

# DRPoly型での正規化・Spolynomial

## 正規化とS多項式の計算

In [30]:
#####################################
def DRPoly_normalization(f, G):
    order = f.order
    flag = True
    while flag:
        flag = False
        for g in G:
            if DRPoly_termOrderCompare(f.ht(), g.ht(), order) == f.ht():
                return_flag, f = f.normal_form(g)
                if return_flag:
                    flag = True
    return f
#####################################            

In [31]:
#####################################
def __DRPoly_normalization(f, G):
    order = f.order
    for g in G:
        if DRPoly_termOrderCompare(f.ht(), g.ht(), order) == f.ht():
            f = f.normal_form(g)
    return f
#####################################            

In [32]:
def DRPoly_Spolynomial(f, g):
    #f, g = DRPoly(f, order), DRPoly(g, order)
    order = f.order
    ht_f, ht_g = f.ht(), g.ht()
    LCM = DRPoly_monomLCM(ht_f, ht_g)
    _f = DRPoly_monomQuo(LCM, f.hm(), order)
    _g = DRPoly_monomQuo(LCM, g.hm(), order)
    Spoly = DRPoly(_f, order) * f - DRPoly(_g, order) * g
    return Spoly

In [33]:
def _DRPoly_BuchbergerAlgorithm(F, order):
    import itertools
    G = F.copy()
    P = list(itertools.combinations(F, 2))
    while len(P) != 0:
        f, g = P[0]
        P.remove((P[0][0], P[0][1]))
        r = DRPoly_normalization(DRPoly_Spolynomial(f, g), G)
        if not r.is_zero():
            for h in G:
                P.append((h, r))
            G.append(r)
    return G

## 動作確認

In [34]:
if __name__ == '__main__':
    """正規化の動作確認"""
    order = 'lex'
    s = Poly(x + 2*y**2*z - 5*z, x, y, z, domain='QQ')
    #s = Poly(x**3 + z**4 - 2, x, y, z, domain=QQ)
    S = DRPoly(s, order)
    g1 = Poly(x**2 + y**2 + z**2 - 4, x, y, z, domain='QQ')
    g2 = Poly(x**2 + 2*y**2 - 5, x, y, z, domain='QQ')
    g3 = Poly(x*z - 1, x, y, z, domain='QQ')
    g4 = Poly(-y**2 + z**2 + 1, x, y, z, domain='QQ')
    g5 = Poly(x + 2*z**3 - 3*z, x, y, z, domain='QQ')
    g = [g1, g2, g3, g4, g5]
    G = [DRPoly(_g, order) for _g in g]
    print(S)
    print(DRPoly_normalization(S, G))

DRPoly(x + 2*y**2*z - 5*z)
DRPoly(0)


In [35]:
if __name__ == '__main__':
    """Spolyの動作確認"""
    order = 'lex'
    f = Poly(x**2 + y**2 + z**2 - 4, (x, y, z), domain=QQ)
    g = Poly(x**2 + 2*y**2 - 5, (x, y, z), domain=QQ)
    h = Poly(x*z - 1, (x, y, z), domain=QQ)
    F = DRPoly(f, order)
    G = DRPoly(g, order)
    H = DRPoly(h, order)
    DRPoly_Spolynomial(F, H)

In [36]:
if __name__ == '__main__':
    """Spolyの動作確認"""
    order = 'lex'
    f1 = (x**2*y + x*y**2 + x*y + 1).as_poly(x, y, domain=ZZ)
    f2 = (x*y - 1).as_poly(x, y, domain=ZZ)
    dr_f1 = DRPoly(f1, order)
    dr_f2 = DRPoly(f2, order)
    print(DRPoly_Spolynomial(dr_f1, dr_f2))

DRPoly(x*y**2 + x*y + x + 1)


In [37]:
if __name__ == '__main__':
    """Buchberger Algorithm"""
    import time
    order = 'lex'
    f = Poly(x**2 + y**2 + z**2 - 4, (x, y, z), domain=QQ)
    g = Poly(x**2 + 2*y**2 -5, (x, y, z), domain=QQ)
    h = Poly(x*z - 1, (x, y, z), domain=QQ)
    _f = DRPoly(f, order)
    _g = DRPoly(g, order)
    _h = DRPoly(h, order)
    F = [_f, _g, _h]
    _DRPoly_BuchbergerAlgorithm(F, order)

# Buchbergerアルゴリズム

## アルゴリズム本体と被約化

In [58]:
def DRPoly_BuchbergerAlgorithm(F, order, criteria='none', strategy='append', test_print=False):
    update = Update(strategy, order)
    Update_function = {'none': update._normal, 'criteria1': update._criteria1, 'criteria2': update._criteria2}
    #strategy_function = {'append': append_strategy, 'normal': normal_strategy, 'suger': sugar_strategy}
    update = Update_function[criteria]
    
    G = [F[0]]
    F = F[1:]
    P, P_LCM, Spoly_sugar_list = [], [], []
    m = 0
    while F != []:
        G.append(F[0])
        F = F[1:]
        P, P_LCM, Spoly_sugar_list = update(P, m, G, P_LCM, Spoly_sugar_list)
        m += 1
        if test_print:
            print('P : ', P)
            if strategy != 'append': print('LCM : ', P_LCM)
            if strategy == 'sugar': print('sugar : ', Spoly_sugar_list)
            print('-----------')
    while P != []:
        f_index, g_index = P[0][0], P[0][1]
        f, g = G[f_index], G[g_index]
        P, P_LCM, Spoly_sugar_list = P[1:], P_LCM[1:], Spoly_sugar_list[1:]
        Spoly = DRPoly_Spolynomial(f, g)
        r = DRPoly_normalization(Spoly, G)
        if not r.is_zero():
            G.append(r)
            P, P_LCM, Spoly_sugar_list = update(P, m, G, P_LCM, Spoly_sugar_list)
            if test_print:
                print('P : ', P)
                if strategy != 'append': print('LCM : ', P_LCM)
                if strategy == 'sugar': print('sugar : ', Spoly_sugar_list)
                print('-----------')
            m += 1
    DRPoly_minimizeAndNormalization(G)
    return G

In [59]:
def DRPoly_minimizeAndNormalization(G):
    # minimize
    """print(G)
    for g in G:
        _G = G.copy()
        _G.remove(g)
        for _g in _G:
            if DRPoly_monomDivisible(_g.ht(), g.ht()):
                print('divide : ', _g, g)
                G.remove(_g)
                print('removed : ', G)
            else:
                print('not divide : ', _g, g)
        print('------------')
    print(G)"""
    removed_list = []
    for g in G:
        if g in removed_list:
            continue
        _G = G.copy()
        _G.remove(g)
        for _g in _G:
            if _g in removed_list:
                continue
            if DRPoly_monomDivisible(_g.ht(), g.ht()):
                removed_list.append(_g)
    for r in removed_list:
        G.remove(r)

    # reduction
    """g_index = -1
    for g in G:
        g_index += 1
        _G = G.copy()
        _G.remove(g)
        normalForm_g = DRPoly_normalization(g, _G)
        normalForm_g = normalForm_g.toPositive()
        G[g_index] = normalForm_g"""
    g_index = -1
    while True:
        g_index += 1
        g = G[g_index]
        _G = G.copy()
        _G.remove(g)
        normalForm_g = DRPoly_normalization(g, _G)
        if normalForm_g.is_zero():
            G.remove(g)
            g_index -= 1
        else:
            normalForm_g = normalForm_g.toPositive()
            G[g_index] = normalForm_g
        if len(G) == g_index+1:
            break
    # sorting
    order = G[0].order
    DRPolyList_sorting(G, order)

## Update

In [60]:
class Update:
    # Spolyが0に正規化されない十分条件（criteria）から，クリティカルペアを予め除去
    def __init__(self, strategy, order):
        self.order = order
        selectionStrategy = SelectionStrategy(order)
        strategy_function = {'append': selectionStrategy._append, 
                             'normal': selectionStrategy._normal, 'sugar': selectionStrategy._sugar}
        self.strategy = strategy_function[strategy]
        self.strategy_key = strategy
        
        
    def _normal(self, P, m, G, P_LCM, Spoly_sugar_list):
        strategy = self.strategy
        #P_LCM, Spoly_sugar_list = [], []
        ht_g_mp1 = G[m+1].ht()
        for i in range(m+1):
            if self.strategy_key == 'normal' or self.strategy_key == 'sugar':
                P_LCM.append(DRPoly_monomLCM(G[i].ht(), ht_g_mp1))
                if self.strategy_key == 'sugar':
                    Spoly_sugar_list.append(DRPoly_Spolynomial(G[i], G[m+1]).sugar)
            P, P_LCM, Spoly_sugar_list = strategy(P, m, i, P_LCM, Spoly_sugar_list)
        return P, P_LCM, Spoly_sugar_list
    
    
    def _criteria1(self, P, m, G, P_LCM, Spoly_sugar_list):
        strategy = self.strategy
        #P_LCM, Spoly_sugar_list = [], []
        ht_g_mp1= G[m+1].ht()
        for i in  range(m+1):
            ht_g_i = G[i].ht()
            if DRPoly_monomLCM(ht_g_i, ht_g_mp1) != ht_g_i * ht_g_mp1:
                #print('criteria1 : ', DRPoly_monomLCM(ht_g_i, ht_g_mp1),  ht_g_i * ht_g_mp1)
                if self.strategy_key == 'normal' or self.strategy_key == 'sugar':
                    P_LCM.append(DRPoly_monomLCM(G[i].ht(), ht_g_mp1))
                    if self.strategy_key == 'sugar':
                        Spoly_sugar_list.append(DRPoly_Spolynomial(G[i], G[m+1]).sugar)
                P, P_LCM, Spoly_sugar_list = strategy(P, m, i, P_LCM, Spoly_sugar_list)
            else:
                pass
                #print('else')
        return P, P_LCM, Spoly_sugar_list
    
        
    def _criteria2(self, P, m, G, P_LCM, Spoly_sugar_list):
        strategy = self.strategy
        #P_LCM, Spoly_sugar_list = [], []
        ht_g_mp1 = G[m+1].ht()
        # B成立を除去
        P_index = -1
        for i, j in P:
            P_index += 1
            ht_g_i = G[i].ht()
            ht_g_j = G[j].ht()
            LCM_ij = DRPoly_monomLCM(ht_g_i, ht_g_j)
            if DRPoly_monomDivisible(LCM_ij, ht_g_mp1):
                if DRPoly_monomLCM(ht_g_i, ht_g_mp1) != LCM_ij:
                    if DRPoly_monomLCM(ht_g_mp1, ht_g_j) != LCM_ij:
                        del P[P_index]
        for i in range(m+1):
            continue_flag = False
            LCM_i_mp1 = DRPoly_monomLCM(G[i].ht(), ht_g_mp1)
            for k in range(i):
                LCM_mp1_k = DRPoly_monomLCM(ht_g_mp1, G[k].ht())
                if LCM_i_mp1 == LCM_mp1_k:
                    # F不成立ペアのみ通過
                    continue_flag = True
                    break
            if continue_flag:
                continue
            for k in range(m+1):
                ht_g_k = G[k].ht()
                LCM_mp1_k = DRPoly_monomLCM(ht_g_mp1, ht_g_k)
                if DRPoly_monomDivisible(LCM_i_mp1, ht_g_k):
                    if LCM_mp1_k != LCM_i_mp1:
                        # M不成立ペアのみ通過
                        continue_flag = True
                        break
            if continue_flag:
                continue_flag = False
                continue
            # Criteria1 Check
            if LCM_i_mp1 != G[i].ht() * ht_g_mp1:
                if self.strategy_key == 'normal' or self.strategy_key == 'sugar':
                    P_LCM.append(LCM_i_mp1)
                    if self.strategy_key == 'sugar':
                        Spoly_sugar_list.append(DRPoly_Spolynomial(G[i], G[m+1]).sugar)
                P, P_LCM, Spoly_sugar_list = strategy(P, m, i, P_LCM, Spoly_sugar_list)
        return P, P_LCM, Spoly_sugar_list

## selection strategy

In [61]:
class SelectionStrategy:
    # ペアができるだけ増えないであろう順番でSpolyを計算（ペアPの順番を並び替える）
    def __init__(self, order):
        self.order = order
    
    
    def _append(self, P, m, i, P_LCM, Spoly_sugar_list):
        P.append((i, m+1))
        return P, P_LCM, Spoly_sugar_list

    
    def _normal(self, P, m, i, P_LCM, Spoly_sugar_list):
        if len(P_LCM) == 1:
            P.insert(i, (i, m+1))
            return P, P_LCM, Spoly_sugar_list
        LCM_i_mp1 = P_LCM[-1]
        del P_LCM[-1]
        for index in range(len(P_LCM)):
            #if LCM_i_mp1 == DRPoly_termOrderCompare(LCM_i_mp1, P_LCM[index], order=self.order):
              #  continue
            #else:
            #print(LCM_i_mp1.List(), P_LCM[index], DRPoly_termOrderCompare(LCM_i_mp1, P_LCM[index], order=self.order))
            if LCM_i_mp1 != DRPoly_termOrderCompare(LCM_i_mp1, P_LCM[index], order=self.order):
                P_LCM.insert(index, LCM_i_mp1)
                P.insert(index, (i, m+1))
                #print('###P : ', P)
                #print('###P_LCM : ', P_LCM, '\n')
                break
        else:
            P_LCM.append(LCM_i_mp1)
            P.append((i, m+1))
            #print('===P : ', P)
            #print('===P_LCM : ', P_LCM, '\n')
        return P, P_LCM, Spoly_sugar_list
    
    
    def _sugar(self, P, m, i, P_LCM, Spoly_sugar_list):
        if len(P_LCM) == 1:
            P.insert(i, (i, m+1))
            return P, P_LCM, Spoly_sugar_list
        LCM_i_mp1 = P_LCM[-1]
        del P_LCM[-1]
        Spoly_sugar_i_mp1 = Spoly_sugar_list[-1]
        del Spoly_sugar_list[-1]
        for index in range(len(Spoly_sugar_list)):
            if Spoly_sugar_i_mp1 > Spoly_sugar_list[index]:
                continue
            elif Spoly_sugar_i_mp1 < Spoly_sugar_list[index]:
                Spoly_sugar_list.insert(index, Spoly_sugar_i_mp1)
                P_LCM.insert(index, LCM_i_mp1)
                P.insert(index, (i, m+1))
                break
            else:
                # Spoly sugar が等しいときは，normal strategy に基づく
                if DRPoly_termOrderCompare(LCM_i_mp1, P_LCM[index], self.order) == P_LCM[index]:
                    Spoly_sugar_list.insert(index, Spoly_sugar_i_mp1)
                    P_LCM.insert(index, LCM_i_mp1)
                    P.insert(index, (i, m+1))
                    break
                else:
                    continue
        else:
            Spoly_sugar_list.append(Spoly_sugar_i_mp1)
            P_LCM.append(LCM_i_mp1)
            P.append((i, m+1))
        return P, P_LCM, Spoly_sugar_list

## 動作確認

In [62]:
if __name__ == '__main__':
    """Buchberger Algorithm"""
    import time
    order = 'grlex'
    f = Poly(x**2 + y**2 + z**2 - 4, (x, y, z), domain=QQ)
    g = Poly(x**2 + 2*y**2 - 5, (x, y, z), domain=QQ)
    h = Poly(x*z - 1, (x, y, z), domain=QQ)
    _f = DRPoly(f, order)
    _g = DRPoly(g, order)
    _h = DRPoly(h, order)
    F = [f, g, h]
    _F = [_f, _g, _h]
    valus = f.gens

In [63]:
if __name__ == '__main__':
    print(DRPoly_BuchbergerAlgorithm(_F, order='lex'))

[DRPoly(2*z**3 + x - 3*z), DRPoly(x**2 + 2*z**2 - 3), DRPoly(x*z - 1), DRPoly(y**2 - z**2 - 1)]


In [64]:
if __name__ == '__main__':
    print(GroebnerBasis(F, order='lex'))

GroebnerBasis([x + 2*z**3 - 3*z, y**2 - z**2 - 1, 2*z**4 - 3*z**2 + 1], x, y, z, domain='ZZ', order='lex')


# SymPyOriginal・SymPyPoly・DRPolyの比較

## Cyclic Polynomial

In [44]:
def cyclicPoly(n, order='lex', DR=True):
    if n <= 5:
        variables = [Symbol('x'), Symbol('y'), Symbol('z'), Symbol('u'), Symbol('w')][:n]
    else:
        variables = [Symbol('x' + str(i)) for i in range(n)]
    F = []
    for k in range(n):
        delta = Integer((k+1)==n)
        sum_f = Poly(0, variables, domain=QQ)
        for i in range(n):
            mul_f = Poly(1, variables, domain=QQ)
            for j in range(i, k+i+1):
                x_index = (j+1)%n - 1
                mul_f *= variables[x_index]
            sum_f += mul_f
            if k == n-1:
                break
        f = sum_f - delta
        F.append(f)
    if not DR: return F
    DRPoly_F = []
    for f in F:
        f = DRPoly(f, order)
        DRPoly_F.append(DRPoly(DRPoly_sorting(f, order), order))
    DRPolyList_sorting(DRPoly_F, order)
    return DRPoly_F

In [45]:
if __name__ == '__main__':
    cyclicPoly(4, 'grlex')

In [46]:
if __name__ == '__main__':
    order = 'lex'
    criteria = 'criteria2'
    strategy = 'sugar'
    C = cyclicPoly(4, order)
    #DRPoly_BuchbergerAlgorithm(C, order, criteria, strategy, test_print=False)

In [47]:
if __name__ == '__main__':
    %%time
    for i in range(10):
        GroebnerBasis(cyclicPoly(4, order='lex', DR=False))

UsageError: Line magic function `%%time` not found.


In [ ]:
if __name__ == '__main__':
    %%time
    for i in range(10):
        DRPoly_BuchbergerAlgorithm(C, order, criteria, strategy, test_print=False)

In [ ]:
if __name__ == '__main__':
    import time
    c4 = cyclicPoly(4, order='lex', DR=False)
    order = 'lex'
    criteria = 'criteria2'
    strategy = 'sugar'
    C = cyclicPoly(4, order)
    n = 10

    t1 = time.process_time()
    for i in range(n):
        _ = GroebnerBasis(c4, order='lex')
    t2 = time.process_time()
    print(t2 - t1)

    t1 = time.process_time()
    for i in range(n):
        _ = DRPoly_BuchbergerAlgorithm(C, order, criteria, strategy, test_print=False)
    t2 = time.process_time()
    print(t2 - t1)

In [ ]:
if __name__ == '__main__':
    _C = cyclicPoly(4, order, DR=False)
    print(GroebnerBasis(_C, order=order))

## 比較

### 時間の比較（cyclic3, 4）

In [ ]:
import SymPyPolyGB_forImport as sp
import time
import IPython
import pandas as pd
import numpy as np

In [ ]:
def displayResult(F, _F, order, loop_num, criteria, strategy, cyclic_n=None, output=True, SP=True):
    if output:
        if cyclic_n:
            print('cyclic' + '(' + str(cyclic_n) + ')')
        if cyclic_n == None:
            for poly in F:
                display(poly.as_expr())
        print('---------------------------')
        print('order : ' + order)
        print('loop : ', loop_num)
        print('criteria : ' + criteria)
        print('strategy : ' + strategy)
        print('----------------------------')
        print('[time]')
    t1 = time.process_time()
    for i in range(loop_num):
        result1 = GroebnerBasis(F, order=order)
    t2 = time.process_time()
    OriginalTime = t2 - t1
    if output:
        print('SymPy Original : ', t2 - t1, '[sec]')

    t1 = time.process_time()
    if SP:
        for i in range(loop_num):
            result2 = sp.myGroebnerBasis(F, variable_order, 
                                   term_order, criteria=criteria, selection_strategy=strategy)
    t2 = time.process_time()
    PolyTime = t2 - t1
    if output:
        print('      SymPy Poly : ', t2 - t1, '[sec]')

    t1 = time.process_time()
    for i in range(loop_num):
        result3 = DRPoly_BuchbergerAlgorithm(_F, order, criteria, strategy)
    t2 = time.process_time()
    DRTime = t2 - t1
    if output:
        print('              DRPoly : ', t2 - t1, '[sec]')
        print('----------------------------')
        print('[result]')
        print('SymPy Original : ', result1)
        print('      SymPy Poly : ', result2)
        print('              DRPoly : ', result3)
    return OriginalTime, PolyTime, DRTime

In [ ]:
if __name__ == '__main__':
    loop_num = 3
    order = 'grevlex'
    variable_order = [x, y, z]
    if order == 'lex': term_order = sp.LexOrder
    if order == 'grlex': term_order = sp.GLexOrder
    if order == 'grevlex': term_order = sp.GrevLexOrder

    # none, criteria1, criteria2
    criteria = 'criteria2'

    # append, normal, sugar
    strategy = 'sugar'

    f = Poly(x**2 + y**2 + z**2 - 4, (x, y, z), domain=QQ)
    g = Poly(x**2 + 2*y**2 -5, (x, y, z), domain=QQ)
    h = Poly(x*z - 1, (x, y, z), domain=QQ)
    _f = DRPoly(f, order)
    _g = DRPoly(g, order)
    _h = DRPoly(h, order)
    F = [f, g, h]
    _F = [_f, _g, _h]
    _1, _2, _3 = displayResult(F, _F, order, loop_num, criteria, strategy)

In [ ]:
if __name__ == '__main__':
    loop_num = 3
    n = 3
    order = 'lex'
    variable_order = [x, y, z]
    if order == 'lex': term_order = sp.LexOrder
    if order == 'grlex': term_order = sp.GLexOrder
    if order == 'grevlex': term_order = sp.GrevLexOrder

    # none, criteria1, criteria2
    criteria = 'criteria2'

    # append, normal, sugar
    strategy = 'sugar'

    F = cyclicPoly(n, DR=False)
    _F = cyclicPoly(n, DR=True)
    _1, _2, _3 = displayResult(F, _F, order, loop_num, criteria, strategy, n)

In [ ]:
if __name__ == '__main__':
    loop_num = 3
    n = 3
    order = 'lex'
    variable_order = [x, y, z]
    if order == 'lex': term_order = sp.LexOrder
    if order == 'grlex': term_order = sp.GLexOrder
    if order == 'grevlex': term_order = sp.GrevLexOrder
    F = cyclicPoly(n, DR=False)
    _F = cyclicPoly(n, DR=True)

    keyPair, O_list, P_list, D_list = [], [], [], []
    for c_key in ('none', 'criteria1', 'criteria2'):
        for s_key in ('append', 'normal', 'sugar'):
            keyPair.append((c_key, s_key))

    for criteria, strategy in keyPair:
        OriginalTime, PolyTime, DRTime = displayResult(F, _F, order, loop_num, criteria, strategy, n, output=False)
        O_list.append(OriginalTime)
        P_list.append(PolyTime)
        D_list.append(DRTime)

    O_list = np.array(O_list).reshape(3, 3)
    P_list = np.array(P_list).reshape(3, 3)
    D_list = np.array(D_list).reshape(3, 3)

In [ ]:
if __name__ == '__main__':
    print('[cyclic(3)]')
    print('SymPy Original')
    display(pd.DataFrame(O_list, columns=['append', 'normal', 'sugar'], index=['none', 'criteria1', 'criteria2']))
    print('SymPy Poly')
    display(pd.DataFrame(P_list, columns=['append', 'normal', 'sugar'], index=['none', 'criteria1', 'criteria2']))
    print('DRPoly')
    display(pd.DataFrame(D_list, columns=['append', 'normal', 'sugar'], index=['none', 'criteria1', 'criteria2']))

In [ ]:
if __name__ == '__main__':
    loop_num = 3
    n = 4
    order = 'lex'
    variable_order = [x, y, z]
    if order == 'lex': term_order = sp.LexOrder
    if order == 'grlex': term_order = sp.GLexOrder
    if order == 'grevlex': term_order = sp.GrevLexOrder
    F = cyclicPoly(n, DR=False)
    _F = cyclicPoly(n, DR=True)

    keyPair, O_list, P_list, D_list = [], [], [], []
    for c_key in ('none', 'criteria1', 'criteria2'):
        for s_key in ('append', 'normal', 'sugar'):
            keyPair.append((c_key, s_key))

    for criteria, strategy in keyPair:
        OriginalTime, PolyTime, DRTime = displayResult(F, _F, order, 
                                                       loop_num, criteria, strategy, n, output=False, SP=False)
        O_list.append(OriginalTime)
        P_list.append(PolyTime)
        D_list.append(DRTime)

    O_list = np.array(O_list).reshape(3, 3)
    P_list = np.array(P_list).reshape(3, 3)
    D_list = np.array(D_list).reshape(3, 3)

In [ ]:
if __name__ == '__main__':
    print('[cyclic(4)]')
    print('SymPy Original')
    display(pd.DataFrame(O_list, columns=['append', 'normal', 'sugar'], index=['none', 'criteria1', 'criteria2']))
    #print('SymPy Poly')
    #display(pd.DataFrame(P_list, columns=['normal', 'sugar'], index=['criteria1', 'criteria2']))
    print('DRPoly')
    display(pd.DataFrame(D_list, columns=['append', 'normal', 'sugar'], index=['none', 'criteria1', 'criteria2']))

### Spolyの呼ばれる回数（cyclic3, 4）

In [ ]:
def DRPoly_BuchbergerAlgorithm_SpolyCall(F, order, criteria='none', strategy='append', max_call=99, test_print=False):
    update = Update(strategy, order)
    Update_function = {'none': update._normal, 'criteria1': update._criteria1, 'criteria2': update._criteria2}
    #strategy_function = {'append': append_strategy, 'normal': normal_strategy, 'suger': sugar_strategy}
    update = Update_function[criteria]
    
    if test_print:
        print('====================')
        print('ciriteria : ', criteria)
        print('strategy : ', strategy)
    G = [F[0]]
    F = F[1:]
    P, P_LCM, Spoly_sugar_list = [], [], []
    m = 0
    while F != []:
        G.append(F[0])
        F = F[1:]
        P, P_LCM, Spoly_sugar_list = update(P, m, G, P_LCM, Spoly_sugar_list)
        m += 1
        if test_print:
            print('P : ', P)
            if strategy != 'append': print('LCM : ', P_LCM)
            if strategy == 'sugar': print('sugar : ', Spoly_sugar_list)
            print('-----------')
    Spoly_call = 0
    while P != []:
        f_index, g_index = P[0][0], P[0][1]
        f, g = G[f_index], G[g_index]
        P, P_LCM, Spoly_sugar_list = P[1:], P_LCM[1:], Spoly_sugar_list[1:]
        Spoly = DRPoly_Spolynomial(f, g)
        Spoly_call += 1
        if Spoly_call == max_call:
            return Spoly_call
        r = DRPoly_normalization(Spoly, G)
        if not r.is_zero():
            G.append(r)
            P, P_LCM, Spoly_sugar_list = update(P, m, G, P_LCM, Spoly_sugar_list)
            if test_print:
                print('P : ', P)
                if strategy != 'append': print('LCM : ', P_LCM)
                if strategy == 'sugar': print('sugar : ', Spoly_sugar_list)
                print('-----------')
            m += 1
    #print('before mini G : ', G)
    DRPoly_minimizeAndNormalization(G)
    #return G
    return Spoly_call

In [ ]:
if __name__ == '__main__':
    loop_num = 3
    n = 3
    order = 'lex'
    variable_order = [x, y, z]
    max_call = 500
    if order == 'lex': term_order = sp.LexOrder
    if order == 'grlex': term_order = sp.GLexOrder
    if order == 'grevlex': term_order = sp.GrevLexOrder
    F = cyclicPoly(n, DR=False)
    _F = cyclicPoly(n, DR=True)

    keyPair, Num_list = [], []
    for c_key in ('none', 'criteria1', 'criteria2'):
        for s_key in ('append', 'normal', 'sugar'):
            keyPair.append((c_key, s_key))

    for criteria, strategy in keyPair:
        SpolyCall = DRPoly_BuchbergerAlgorithm_SpolyCall(_F, order, criteria, strategy, max_call, test_print=False)
        Num_list.append(SpolyCall)

    Num_list = np.array(Num_list).reshape(3, 3)

In [ ]:
if __name__ == '__main__':
    print('cyclic(3)')
    display(pd.DataFrame(Num_list, columns=['append', 'normal', 'sugar'], index=['none', 'criteria1', 'criteria2']))

In [ ]:
if __name__ == '__main__':
    loop_num = 3
    n = 4
    order = 'lex'
    variable_order = [x, y, z]
    max_call = 500
    if order == 'lex': term_order = sp.LexOrder
    if order == 'grlex': term_order = sp.GLexOrder
    if order == 'grevlex': term_order = sp.GrevLexOrder
    F = cyclicPoly(n, DR=False)
    _F = cyclicPoly(n, DR=True)

    keyPair, Num_list = [], []
    for c_key in ('none', 'criteria1', 'criteria2'):
        for s_key in ('append', 'normal', 'sugar'):
            keyPair.append((c_key, s_key))

    for criteria, strategy in keyPair:
        SpolyCall = DRPoly_BuchbergerAlgorithm_SpolyCall(_F, order, criteria, strategy, max_call)
        Num_list.append(SpolyCall)

    Num_list = np.array(Num_list).reshape(3, 3)

In [ ]:
if __name__ == '__main__':
    print('cyclic(4)')

    display(pd.DataFrame(Num_list, columns=['append', 'normal', 'sugar'], index=['none', 'criteria1', 'criteria2']))

In [ ]:
"""loop_num = 1
n = 5
order = 'lex'
variable_order = [x, y, z]
max_call = 500
if order == 'lex': term_order = sp.LexOrder
if order == 'grlex': term_order = sp.GLexOrder
if order == 'grevlex': term_order = sp.GrevLexOrder
F = cyclicPoly(n, DR=False)
_F = cyclicPoly(n, DR=True)

keyPair, Num_list = [], []
for c_key in ('none', 'criteria1', 'criteria2'):
    for s_key in ('append', 'normal', 'sugar'):
        keyPair.append((c_key, s_key))

for criteria, strategy in keyPair:
    SpolyCall = DRPoly_BuchbergerAlgorithm_SpolyCall(_F, order, criteria, strategy, max_call)
    Num_list.append(SpolyCall)

Num_list = np.array(Num_list).reshape(3, 3)"""

In [ ]:
if __name__ == '__main__':
    print('cyclic(5)')
    display(pd.DataFrame(Num_list, columns=['append', 'normal', 'sugar'], index=['none', 'criteria1', 'criteria2']))

In [ ]:
"""Groebner bases algorithms. """

from __future__ import print_function, division

from sympy.core.symbol import Dummy
from sympy.polys.monomials import monomial_mul, monomial_lcm, monomial_divides, term_div
from sympy.polys.orderings import lex
from sympy.polys.polyerrors import DomainError
from sympy.polys.polyconfig import query

def groebner(seq, ring, method=None):
    """
    Computes Groebner basis for a set of polynomials in `K[X]`.
    Wrapper around the (default) improved Buchberger and the other algorithms
    for computing Groebner bases. The choice of algorithm can be changed via
    ``method`` argument or :func:`sympy.polys.polyconfig.setup`, where
    ``method`` can be either ``buchberger`` or ``f5b``.
    """
    if method is None:
        method = query('groebner')

    _groebner_methods = {
        'buchberger': _buchberger,
        'f5b': _f5b,
    }

    try:
        _groebner = _groebner_methods[method]
    except KeyError:
        raise ValueError("'%s' is not a valid Groebner bases algorithm (valid are 'buchberger' and 'f5b')" % method)

    domain, orig = ring.domain, None

    if not domain.is_Field or not domain.has_assoc_Field:
        try:
            orig, ring = ring, ring.clone(domain=domain.get_field())
        except DomainError:
            raise DomainError("can't compute a Groebner basis over %s" % domain)
        else:
            seq = [ s.set_ring(ring) for s in seq ]

    G = _groebner(seq, ring)

    if orig is not None:
        G = [ g.clear_denoms()[1].set_ring(orig) for g in G ]

    return G

def _buchberger(f, ring):
    """
    Computes Groebner basis for a set of polynomials in `K[X]`.
    Given a set of multivariate polynomials `F`, finds another
    set `G`, such that Ideal `F = Ideal G` and `G` is a reduced
    Groebner basis.
    The resulting basis is unique and has monic generators if the
    ground domains is a field. Otherwise the result is non-unique
    but Groebner bases over e.g. integers can be computed (if the
    input polynomials are monic).
    Groebner bases can be used to choose specific generators for a
    polynomial ideal. Because these bases are unique you can check
    for ideal equality by comparing the Groebner bases.  To see if
    one polynomial lies in an ideal, divide by the elements in the
    base and see if the remainder vanishes.
    They can also be used to solve systems of polynomial equations
    as,  by choosing lexicographic ordering,  you can eliminate one
    variable at a time, provided that the ideal is zero-dimensional
    (finite number of solutions).
    Notes
    =====
    Algorithm used: an improved version of Buchberger's algorithm
    as presented in T. Becker, V. Weispfenning, Groebner Bases: A
    Computational Approach to Commutative Algebra, Springer, 1993,
    page 232.
    References
    ==========
    .. [1] [Bose03]_
    .. [2] [Giovini91]_
    .. [3] [Ajwa95]_
    .. [4] [Cox97]_
    """
    order = ring.order

    monomial_mul = ring.monomial_mul
    monomial_div = ring.monomial_div
    monomial_lcm = ring.monomial_lcm

    def select(P):
        # normal selection strategy
        # select the pair with minimum LCM(LM(f), LM(g))
        pr = min(P, key=lambda pair: order(monomial_lcm(f[pair[0]].LM, f[pair[1]].LM)))
        return pr

    def normal(g, J):
        h = g.rem([ f[j] for j in J ])

        if not h:
            return None
        else:
            h = h.monic()

            if not h in I:
                I[h] = len(f)
                f.append(h)

            return h.LM, I[h]

    def update(G, B, ih):
        # update G using the set of critical pairs B and h
        # [BW] page 230
        h = f[ih]
        mh = h.LM

        # filter new pairs (h, g), g in G
        C = G.copy()
        D = set()

        while C:
            # select a pair (h, g) by popping an element from C
            ig = C.pop()
            g = f[ig]
            mg = g.LM
            LCMhg = monomial_lcm(mh, mg)

            def lcm_divides(ip):
                # LCM(LM(h), LM(p)) divides LCM(LM(h), LM(g))
                m = monomial_lcm(mh, f[ip].LM)
                return monomial_div(LCMhg, m)

            # HT(h) and HT(g) disjoint: mh*mg == LCMhg
            if monomial_mul(mh, mg) == LCMhg or (
                not any(lcm_divides(ipx) for ipx in C) and
                    not any(lcm_divides(pr[1]) for pr in D)):
                D.add((ih, ig))

        E = set()

        while D:
            # select h, g from D (h the same as above)
            ih, ig = D.pop()
            mg = f[ig].LM
            LCMhg = monomial_lcm(mh, mg)

            if not monomial_mul(mh, mg) == LCMhg:
                E.add((ih, ig))

        # filter old pairs
        B_new = set()

        while B:
            # select g1, g2 from B (-> CP)
            ig1, ig2 = B.pop()
            mg1 = f[ig1].LM
            mg2 = f[ig2].LM
            LCM12 = monomial_lcm(mg1, mg2)

            # if HT(h) does not divide lcm(HT(g1), HT(g2))
            if not monomial_div(LCM12, mh) or \
                monomial_lcm(mg1, mh) == LCM12 or \
                    monomial_lcm(mg2, mh) == LCM12:
                B_new.add((ig1, ig2))

        B_new |= E

        # filter polynomials
        G_new = set()

        while G:
            ig = G.pop()
            mg = f[ig].LM

            if not monomial_div(mg, mh):
                G_new.add(ig)

        G_new.add(ih)

        return G_new, B_new
        # end of update ################################

    if not f:
        return []

    # replace f with a reduced list of initial polynomials; see [BW] page 203
    f1 = f[:]

    while True:
        f = f1[:]
        f1 = []

        for i in range(len(f)):
            p = f[i]
            r = p.rem(f[:i])

            if r:
                f1.append(r.monic())

        if f == f1:
            break

    I = {}            # ip = I[p]; p = f[ip]
    F = set()         # set of indices of polynomials
    G = set()         # set of indices of intermediate would-be Groebner basis
    CP = set()        # set of pairs of indices of critical pairs

    for i, h in enumerate(f):
        I[h] = i
        F.add(i)

    #####################################
    # algorithm GROEBNERNEWS2 in [BW] page 232

    while F:
        # select p with minimum monomial according to the monomial ordering
        h = min([f[x] for x in F], key=lambda f: order(f.LM))
        ih = I[h]
        F.remove(ih)
        G, CP = update(G, CP, ih)

    # count the number of critical pairs which reduce to zero
    reductions_to_zero = 0

    while CP:
        ig1, ig2 = select(CP)
        CP.remove((ig1, ig2))

        h = spoly(f[ig1], f[ig2], ring)
        # ordering divisors is on average more efficient [Cox] page 111
        G1 = sorted(G, key=lambda g: order(f[g].LM))
        ht = normal(h, G1)

        if ht:
            G, CP = update(G, CP, ht[1])
        else:
            reductions_to_zero += 1

    ######################################
    # now G is a Groebner basis; reduce it
    Gr = set()

    for ig in G:
        ht = normal(f[ig], G - set([ig]))

        if ht:
            Gr.add(ht[1])

    Gr = [f[ig] for ig in Gr]

    # order according to the monomial ordering
    Gr = sorted(Gr, key=lambda f: order(f.LM), reverse=True)

    return Gr

def spoly(p1, p2, ring):
    """
    Compute LCM(LM(p1), LM(p2))/LM(p1)*p1 - LCM(LM(p1), LM(p2))/LM(p2)*p2
    This is the S-poly provided p1 and p2 are monic
    """
    LM1 = p1.LM
    LM2 = p2.LM
    LCM12 = ring.monomial_lcm(LM1, LM2)
    m1 = ring.monomial_div(LCM12, LM1)
    m2 = ring.monomial_div(LCM12, LM2)
    s1 = p1.mul_monom(m1)
    s2 = p2.mul_monom(m2)
    s = s1 - s2
    return s

# F5B

# convenience functions


def Sign(f):
    return f[0]


def Polyn(f):
    return f[1]


def Num(f):
    return f[2]


def sig(monomial, index):
    return (monomial, index)


def lbp(signature, polynomial, number):
    return (signature, polynomial, number)

# signature functions


def sig_cmp(u, v, order):
    """
    Compare two signatures by extending the term order to K[X]^n.
    u < v iff
        - the index of v is greater than the index of u
    or
        - the index of v is equal to the index of u and u[0] < v[0] w.r.t. order
    u > v otherwise
    """
    if u[1] > v[1]:
        return -1
    if u[1] == v[1]:
        #if u[0] == v[0]:
        #    return 0
        if order(u[0]) < order(v[0]):
            return -1
    return 1


def sig_key(s, order):
    """
    Key for comparing two signatures.
    s = (m, k), t = (n, l)
    s < t iff [k > l] or [k == l and m < n]
    s > t otherwise
    """
    return (-s[1], order(s[0]))


def sig_mult(s, m):
    """
    Multiply a signature by a monomial.
    The product of a signature (m, i) and a monomial n is defined as
    (m * t, i).
    """
    return sig(monomial_mul(s[0], m), s[1])

# labeled polynomial functions


def lbp_sub(f, g):
    """
    Subtract labeled polynomial g from f.
    The signature and number of the difference of f and g are signature
    and number of the maximum of f and g, w.r.t. lbp_cmp.
    """
    if sig_cmp(Sign(f), Sign(g), Polyn(f).ring.order) < 0:
        max_poly = g
    else:
        max_poly = f

    ret = Polyn(f) - Polyn(g)

    return lbp(Sign(max_poly), ret, Num(max_poly))


def lbp_mul_term(f, cx):
    """
    Multiply a labeled polynomial with a term.
    The product of a labeled polynomial (s, p, k) by a monomial is
    defined as (m * s, m * p, k).
    """
    return lbp(sig_mult(Sign(f), cx[0]), Polyn(f).mul_term(cx), Num(f))


def lbp_cmp(f, g):
    """
    Compare two labeled polynomials.
    f < g iff
        - Sign(f) < Sign(g)
    or
        - Sign(f) == Sign(g) and Num(f) > Num(g)
    f > g otherwise
    """
    if sig_cmp(Sign(f), Sign(g), Polyn(f).ring.order) == -1:
        return -1
    if Sign(f) == Sign(g):
        if Num(f) > Num(g):
            return -1
        #if Num(f) == Num(g):
        #    return 0
    return 1


def lbp_key(f):
    """
    Key for comparing two labeled polynomials.
    """
    return (sig_key(Sign(f), Polyn(f).ring.order), -Num(f))

# algorithm and helper functions


def critical_pair(f, g, ring):
    """
    Compute the critical pair corresponding to two labeled polynomials.
    A critical pair is a tuple (um, f, vm, g), where um and vm are
    terms such that um * f - vm * g is the S-polynomial of f and g (so,
    wlog assume um * f > vm * g).
    For performance sake, a critical pair is represented as a tuple
    (Sign(um * f), um, f, Sign(vm * g), vm, g), since um * f creates
    a new, relatively expensive object in memory, whereas Sign(um *
    f) and um are lightweight and f (in the tuple) is a reference to
    an already existing object in memory.
    """
    domain = ring.domain

    ltf = Polyn(f).LT
    ltg = Polyn(g).LT
    lt = (monomial_lcm(ltf[0], ltg[0]), domain.one)

    um = term_div(lt, ltf, domain)
    vm = term_div(lt, ltg, domain)

    # The full information is not needed (now), so only the product
    # with the leading term is considered:
    fr = lbp_mul_term(lbp(Sign(f), Polyn(f).leading_term(), Num(f)), um)
    gr = lbp_mul_term(lbp(Sign(g), Polyn(g).leading_term(), Num(g)), vm)

    # return in proper order, such that the S-polynomial is just
    # u_first * f_first - u_second * f_second:
    if lbp_cmp(fr, gr) == -1:
        return (Sign(gr), vm, g, Sign(fr), um, f)
    else:
        return (Sign(fr), um, f, Sign(gr), vm, g)


def cp_cmp(c, d):
    """
    Compare two critical pairs c and d.
    c < d iff
        - lbp(c[0], _, Num(c[2]) < lbp(d[0], _, Num(d[2])) (this
        corresponds to um_c * f_c and um_d * f_d)
    or
        - lbp(c[0], _, Num(c[2]) >< lbp(d[0], _, Num(d[2])) and
        lbp(c[3], _, Num(c[5])) < lbp(d[3], _, Num(d[5])) (this
        corresponds to vm_c * g_c and vm_d * g_d)
    c > d otherwise
    """
    zero = Polyn(c[2]).ring.zero

    c0 = lbp(c[0], zero, Num(c[2]))
    d0 = lbp(d[0], zero, Num(d[2]))

    r = lbp_cmp(c0, d0)

    if r == -1:
        return -1
    if r == 0:
        c1 = lbp(c[3], zero, Num(c[5]))
        d1 = lbp(d[3], zero, Num(d[5]))

        r = lbp_cmp(c1, d1)

        if r == -1:
            return -1
        #if r == 0:
        #    return 0
    return 1


def cp_key(c, ring):
    """
    Key for comparing critical pairs.
    """
    return (lbp_key(lbp(c[0], ring.zero, Num(c[2]))), lbp_key(lbp(c[3], ring.zero, Num(c[5]))))


def s_poly(cp):
    """
    Compute the S-polynomial of a critical pair.
    The S-polynomial of a critical pair cp is cp[1] * cp[2] - cp[4] * cp[5].
    """
    return lbp_sub(lbp_mul_term(cp[2], cp[1]), lbp_mul_term(cp[5], cp[4]))


def is_rewritable_or_comparable(sign, num, B):
    """
    Check if a labeled polynomial is redundant by checking if its
    signature and number imply rewritability or comparability.
    (sign, num) is comparable if there exists a labeled polynomial
    h in B, such that sign[1] (the index) is less than Sign(h)[1]
    and sign[0] is divisible by the leading monomial of h.
    (sign, num) is rewritable if there exists a labeled polynomial
    h in B, such thatsign[1] is equal to Sign(h)[1], num < Num(h)
    and sign[0] is divisible by Sign(h)[0].
    """
    for h in B:
        # comparable
        if sign[1] < Sign(h)[1]:
            if monomial_divides(Polyn(h).LM, sign[0]):
                return True

        # rewritable
        if sign[1] == Sign(h)[1]:
            if num < Num(h):
                if monomial_divides(Sign(h)[0], sign[0]):
                    return True
    return False


def f5_reduce(f, B):
    """
    F5-reduce a labeled polynomial f by B.
    Continuously searches for non-zero labeled polynomial h in B, such
    that the leading term lt_h of h divides the leading term lt_f of
    f and Sign(lt_h * h) < Sign(f). If such a labeled polynomial h is
    found, f gets replaced by f - lt_f / lt_h * h. If no such h can be
    found or f is 0, f is no further F5-reducible and f gets returned.
    A polynomial that is reducible in the usual sense need not be
    F5-reducible, e.g.:
    >>> from sympy.polys.groebnertools import lbp, sig, f5_reduce, Polyn
    >>> from sympy.polys import ring, QQ, lex
    >>> R, x,y,z = ring("x,y,z", QQ, lex)
    >>> f = lbp(sig((1, 1, 1), 4), x, 3)
    >>> g = lbp(sig((0, 0, 0), 2), x, 2)
    >>> Polyn(f).rem([Polyn(g)])
    0
    >>> f5_reduce(f, [g])
    (((1, 1, 1), 4), x, 3)
    """
    order = Polyn(f).ring.order
    domain = Polyn(f).ring.domain

    if not Polyn(f):
        return f

    while True:
        g = f

        for h in B:
            if Polyn(h):
                if monomial_divides(Polyn(h).LM, Polyn(f).LM):
                    t = term_div(Polyn(f).LT, Polyn(h).LT, domain)
                    if sig_cmp(sig_mult(Sign(h), t[0]), Sign(f), order) < 0:
                        # The following check need not be done and is in general slower than without.
                        #if not is_rewritable_or_comparable(Sign(gp), Num(gp), B):
                        hp = lbp_mul_term(h, t)
                        f = lbp_sub(f, hp)
                        break

        if g == f or not Polyn(f):
            return f


def _f5b(F, ring):
    """
    Computes a reduced Groebner basis for the ideal generated by F.
    f5b is an implementation of the F5B algorithm by Yao Sun and
    Dingkang Wang. Similarly to Buchberger's algorithm, the algorithm
    proceeds by computing critical pairs, computing the S-polynomial,
    reducing it and adjoining the reduced S-polynomial if it is not 0.
    Unlike Buchberger's algorithm, each polynomial contains additional
    information, namely a signature and a number. The signature
    specifies the path of computation (i.e. from which polynomial in
    the original basis was it derived and how), the number says when
    the polynomial was added to the basis.  With this information it
    is (often) possible to decide if an S-polynomial will reduce to
    0 and can be discarded.
    Optimizations include: Reducing the generators before computing
    a Groebner basis, removing redundant critical pairs when a new
    polynomial enters the basis and sorting the critical pairs and
    the current basis.
    Once a Groebner basis has been found, it gets reduced.
    References
    ==========
    .. [1] Yao Sun, Dingkang Wang: "A New Proof for the Correctness of F5
           (F5-Like) Algorithm", http://arxiv.org/abs/1004.0084 (specifically
           v4)
    .. [2] Thomas Becker, Volker Weispfenning, Groebner bases: A computational
           approach to commutative algebra, 1993, p. 203, 216
    """
    order = ring.order

    # reduce polynomials (like in Mario Pernici's implementation) (Becker, Weispfenning, p. 203)
    B = F
    while True:
        F = B
        B = []

        for i in range(len(F)):
            p = F[i]
            r = p.rem(F[:i])

            if r:
                B.append(r)

        if F == B:
            break

    # basis
    B = [lbp(sig(ring.zero_monom, i + 1), F[i], i + 1) for i in range(len(F))]
    B.sort(key=lambda f: order(Polyn(f).LM), reverse=True)

    # critical pairs
    CP = [critical_pair(B[i], B[j], ring) for i in range(len(B)) for j in range(i + 1, len(B))]
    CP.sort(key=lambda cp: cp_key(cp, ring), reverse=True)

    k = len(B)

    reductions_to_zero = 0

    while len(CP):
        cp = CP.pop()

        # discard redundant critical pairs:
        if is_rewritable_or_comparable(cp[0], Num(cp[2]), B):
            continue
        if is_rewritable_or_comparable(cp[3], Num(cp[5]), B):
            continue

        s = s_poly(cp)

        p = f5_reduce(s, B)

        p = lbp(Sign(p), Polyn(p).monic(), k + 1)

        if Polyn(p):
            # remove old critical pairs, that become redundant when adding p:
            indices = []
            for i, cp in enumerate(CP):
                if is_rewritable_or_comparable(cp[0], Num(cp[2]), [p]):
                    indices.append(i)
                elif is_rewritable_or_comparable(cp[3], Num(cp[5]), [p]):
                    indices.append(i)

            for i in reversed(indices):
                del CP[i]

            # only add new critical pairs that are not made redundant by p:
            for g in B:
                if Polyn(g):
                    cp = critical_pair(p, g, ring)
                    if is_rewritable_or_comparable(cp[0], Num(cp[2]), [p]):
                        continue
                    elif is_rewritable_or_comparable(cp[3], Num(cp[5]), [p]):
                        continue

                    CP.append(cp)

            # sort (other sorting methods/selection strategies were not as successful)
            CP.sort(key=lambda cp: cp_key(cp, ring), reverse=True)

            # insert p into B:
            m = Polyn(p).LM
            if order(m) <= order(Polyn(B[-1]).LM):
                B.append(p)
            else:
                for i, q in enumerate(B):
                    if order(m) > order(Polyn(q).LM):
                        B.insert(i, p)
                        break

            k += 1

            #print(len(B), len(CP), "%d critical pairs removed" % len(indices))
        else:
            reductions_to_zero += 1

    # reduce Groebner basis:
    H = [Polyn(g).monic() for g in B]
    H = red_groebner(H, ring)

    return sorted(H, key=lambda f: order(f.LM), reverse=True)


def red_groebner(G, ring):
    """
    Compute reduced Groebner basis, from BeckerWeispfenning93, p. 216
    Selects a subset of generators, that already generate the ideal
    and computes a reduced Groebner basis for them.
    """
    def reduction(P):
        """
        The actual reduction algorithm.
        """
        Q = []
        for i, p in enumerate(P):
            h = p.rem(P[:i] + P[i + 1:])
            if h:
                Q.append(h)

        return [p.monic() for p in Q]

    F = G
    H = []

    while F:
        f0 = F.pop()

        if not any(monomial_divides(f.LM, f0.LM) for f in F + H):
            H.append(f0)

    # Becker, Weispfenning, p. 217: H is Groebner basis of the ideal generated by G.
    return reduction(H)


def is_groebner(G, ring):
    """
    Check if G is a Groebner basis.
    """
    for i in range(len(G)):
        for j in range(i + 1, len(G)):
            s = spoly(G[i], G[j], ring)
            s = s.rem(G)
            if s:
                return False

    return True


def is_minimal(G, ring):
    """
    Checks if G is a minimal Groebner basis.
    """
    order = ring.order
    domain = ring.domain

    G.sort(key=lambda g: order(g.LM))

    for i, g in enumerate(G):
        if g.LC != domain.one:
            return False

        for h in G[:i] + G[i + 1:]:
            if monomial_divides(h.LM, g.LM):
                return False

    return True


def is_reduced(G, ring):
    """
    Checks if G is a reduced Groebner basis.
    """
    order = ring.order
    domain = ring.domain

    G.sort(key=lambda g: order(g.LM))

    for i, g in enumerate(G):
        if g.LC != domain.one:
            return False

        for term in g.terms():
            for h in G[:i] + G[i + 1:]:
                if monomial_divides(h.LM, term[0]):
                    return False

    return True

def groebner_lcm(f, g):
    """
    Computes LCM of two polynomials using Groebner bases.
    The LCM is computed as the unique generator of the intersection
    of the two ideals generated by `f` and `g`. The approach is to
    compute a Groebner basis with respect to lexicographic ordering
    of `t*f` and `(1 - t)*g`, where `t` is an unrelated variable and
    then filtering out the solution that doesn't contain `t`.
    References
    ==========
    .. [1] [Cox97]_
    """
    if f.ring != g.ring:
        raise ValueError("Values should be equal")

    ring = f.ring
    domain = ring.domain

    if not f or not g:
        return ring.zero

    if len(f) <= 1 and len(g) <= 1:
        monom = monomial_lcm(f.LM, g.LM)
        coeff = domain.lcm(f.LC, g.LC)
        return ring.term_new(monom, coeff)

    fc, f = f.primitive()
    gc, g = g.primitive()

    lcm = domain.lcm(fc, gc)

    f_terms = [ ((1,) + monom, coeff) for monom, coeff in f.terms() ]
    g_terms = [ ((0,) + monom, coeff) for monom, coeff in g.terms() ] \
            + [ ((1,) + monom,-coeff) for monom, coeff in g.terms() ]

    t = Dummy("t")
    t_ring = ring.clone(symbols=(t,) + ring.symbols, order=lex)

    F = t_ring.from_terms(f_terms)
    G = t_ring.from_terms(g_terms)

    basis = groebner([F, G], t_ring)

    def is_independent(h, j):
        return all(not monom[j] for monom in h.monoms())

    H = [ h for h in basis if is_independent(h, 0) ]

    h_terms = [ (monom[1:], coeff*lcm) for monom, coeff in H[0].terms() ]
    h = ring.from_terms(h_terms)

    return h

def groebner_gcd(f, g):
    """Computes GCD of two polynomials using Groebner bases. """
    if f.ring != g.ring:
        raise ValueError("Values should be equal")
    domain = f.ring.domain

    if not domain.is_Field:
        fc, f = f.primitive()
        gc, g = g.primitive()
        gcd = domain.gcd(fc, gc)

    H = (f*g).quo([groebner_lcm(f, g)])

    if len(H) != 1:
        raise ValueError("Length should be 1")
    h = H[0]

    if not domain.is_Field:
        return gcd*h
    else:
        return h.monic()
